<a href="https://colab.research.google.com/github/Existanze54/sirius-neural-networks-2024/blob/main/Practices/S11_AutoML_LLM/4_LightAutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Краткий обзор AutoML иструментов в 2024 году: <a href='https://habr.com/ru/articles/811425/'>link</a>

<center><img src="https://github.com/Existanze54/sirius-neural-networks-2024/blob/main/Images/LAMA.png?raw=true" width=600></img></center>

LightAutoML github репозиторий: <a href='https://github.com/sb-ai-lab/LightAutoML'>link</a>

LightAutoML (LAMA) – мощный open-source AutoML фреймворк за которым стоит одна из сильнейших по экспертизе DS команд из Sber AI Lab. Суперсила LAMA – это бленды и настраиваемые эксперименты. В то же время LAMA скорее скальпель для профессионалов,. Давно не было обновления, очень надеюсь, что мы увидим его в ближайшее время.

In [ ]:
!pip install lightautoml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.4/416.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.7/215.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 60.9

# Import Libraries & Set Parameters

In [ ]:
# Standard python libraries
import os
import time
import requests

# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, accuracy_score
import torch

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco

Here we setup some parameters to use in the kernel:
- `N_THREADS` - number of vCPUs for LightAutoML model creation
- `N_FOLDS` - number of folds in LightAutoML inner CV
- `RANDOM_STATE` - random seed for better reproducibility
- `TEST_SIZE` - houldout data part size
- `TIMEOUT` - limit in seconds for model to train
- `TARGET_NAME` - target column name in dataset

In [ ]:
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
#TEST_SIZE = 0.2
#TIMEOUT = 10*3600
TIMEOUT = 5*60
TARGET_NAME = 'hospital_death'

In [ ]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

# Load Data

It is important to note that missing values (NaN and other) in the data should be left as is, unless the reason for their presence or their specific meaning are known. Otherwise, AutoML model will perceive the filled NaNs as a true pattern between the data and the target variable, without knowledge and assumptions about missing values, which can negatively affect the model quality. LighAutoML can deal with missing values and outliers automatically.

In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Existanze54/sirius-neural-networks-2024/main/Datasets/patient-survival-prediction/train_preprocessed.csv')
print(train_df.shape)
train_df.head()

(44939, 83)


,Unnamed: 0,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,icu_admit_source,icu_id,...,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,hospital_death
0,0,118.0,69.9,25.719814,0.0,2.0,0.0,162.6,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,4.0,1
1,1,185.0,57.0,20.357278,0.0,0.0,1.0,182.9,0.0,687.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0
2,2,99.0,71.0,30.558683,0.0,5.0,1.0,175.2,0.0,514.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,21.0,75.0,44.990982,0.0,2.0,1.0,175.2,3.0,504.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,4.0,0
4,4,70.0,62.0,16.620499,0.0,0.0,0.0,152.0,0.0,464.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
test_df = pd.read_csv('https://raw.githubusercontent.com/Existanze54/sirius-neural-networks-2024/main/Datasets/patient-survival-prediction/test_preprocessed.csv')
print(test_df.shape)
test_df.head()

(19260, 83)


,Unnamed: 0,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,icu_admit_source,icu_id,...,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,hospital_death
0,0,188.0,69.0,29.605976,0.0,2.0,0.0,165.1,0.0,840.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0
1,1,10.0,68.0,27.986953,0.0,2.0,1.0,185.4,0.0,428.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,4.0,0
2,2,176.0,55.0,32.641470,1.0,2.0,0.0,162.6,2.0,611.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0
3,3,19.0,53.0,19.444444,0.0,2.0,1.0,180.0,3.0,653.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0
4,4,128.0,74.0,16.508909,0.0,2.0,0.0,165.1,1.0,377.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0


## Task definition

First we need to create ```Task``` object - the class to setup what task LightAutoML model should solve with specific loss and metric if necessary (more info can be found [here](https://lightautoml.readthedocs.io/en/latest/pages/modules/generated/lightautoml.tasks.base.Task.html#lightautoml.tasks.base.Task) in our documentation).

The following task types are available:

- ```'binary'``` - for binary classification.

- ```'reg’``` - for regression.

- ```‘multiclass’``` - for multiclass classification.

- ```'multi:reg``` - for multiple regression.

- ```'multilabel'``` - for multi-label classification.

In this example we will consider a binary classification:

In [ ]:
task = Task('binary')

Note that only logloss loss is available for binary task and it is the default loss. Default metric for binary classification is ROC-AUC. See more info about available and default losses and metrics [here](https://lightautoml.readthedocs.io/en/latest/pages/modules/generated/lightautoml.tasks.base.Task.html#lightautoml.tasks.base.Task).

**Depending on the task, you can and shold choose exactly those metrics and losses that you want and need to optimize.**

To solve the task, we need to setup columns roles. LightAutoML can automatically define types and roles of data columns, but it is possible to specify it directly through the dictionary parameter ```roles``` when training AutoML model (see next section "AutoML training"). Specific roles can be specified using a string with the name (any role can be set like this).  So the key in dictionary must be the name of the role, the value must be a list of the names of the corresponding columns in dataset. The **only role you must setup is** ```'target'``` **role** (that is column with target variable obviously), everything else (```'drop', 'numeric', 'categorical', 'group', 'weights'``` etc) is up to user:

In [ ]:
roles = {
    'target': TARGET_NAME,
    'drop': ['patient_id', 'encounter_id']
}

You can also optionally specify the following roles:

- ```'numeric'``` - numerical feature

- ```'category'``` - categorical feature

- ```'text'``` - text data

- ```'datetime'``` - features with date and time

- ```'date'``` - features with date only

- ```'group'``` - features by which the data can be divided into groups and which can be taken into account for group k-fold validation (so the same group is not represented in both testing and training sets)

- ```'drop'``` - features to drop, they will not be used in model building

- ```'weights'``` - object weights for the loss and metric

- ```'path'``` - image file paths (for CV tasks)

- ```'treatment'``` - object group in uplift modelling tasks: treatment or control

Note that role name can be written in any case. Also it is possible to pass individual objects of role classes with specific arguments instead of strings with role names for specific tasks and more optimal pipeline construction ([more details](https://github.com/sb-ai-lab/LightAutoML/blob/master/lightautoml/dataset/roles.py)).

For example, to set the date role, you can use the ```DatetimeRole``` class.

In [ ]:
# from lightautoml.dataset.roles import DatetimeRole

Different seasonality can be extracted from the data through the ```seasonality``` parameter: years (```'y'```), months (```'m'```), days (```'d'```), weekdays (```'wd'```), hours (```'hour'```), minutes (```'min'```), seconds (```'sec'```), milliseconds (```'ms'```), nanoseconds (```'ns'```). This features will be considered as categorical. Another important parameter is ```base_date```. It allows to specify the base date and convert the feature to the distances to this date (set to ```False``` by default). Also for all roles classes there is a ```force_input``` parameter, and if it is ```True```, then the corresponding features will pass all further feature selections and won't be excluded (equals ```False``` by default). Also it is always possible to specify data type for all roles using ```dtype``` argument.

Here is an example of such a role assignment through a class object for date feature (but there is no such feature in the considered dataset):

In [ ]:
# roles = {
#     DatetimeRole(base_date=False, seasonality=('d', 'wd', 'hour')): 'date_time'
# }

Next we are going to create LightAutoML model with `TabularAutoML` class - preset with default model structure in just several lines.

Let's discuss some of the params we can setup:
- `task` - the type of the ML task (the only **must have** parameter)
- `timeout` - time limit in seconds for model to train
- `cpu_limit` - vCPU count for model to use
- `reader_params` - parameter change for ```Reader``` object inside preset, which works on the first step of data preparation: automatic feature typization, preliminary almost-constant features, correct CV setup etc. For example, we setup `n_jobs` threads for typization algo, `cv` folds and `random_state` as inside CV seed.
- `general_params` - general parameters dictionary, in which it is possible to specify a list of algorithms used (```'use_algos'```), nested CV using (```'nested_cv'```) etc.

**Important note**: `reader_params` key is one of the YAML config keys, which is used inside `TabularAutoML` preset. [More details](https://github.com/sb-ai-lab/LightAutoML/blob/master/lightautoml/automl/presets/tabular_config.yml) on its structure with explanation comments can be found on the link attached. Each key from this config can be modified with user settings during preset object initialization. To get more info about different parameters setting (for example, ML algos which can be used in `general_params->use_algos`) please take a look at our [article on TowardsDataScience](https://towardsdatascience.com/lightautoml-preset-usage-tutorial-2cce7da6f936).

Moreover, to receive the automatic report for our model we will use `ReportDeco` decorator and work with the decorated version in the same way as we do with usual one.

In [ ]:
automl = TabularUtilizedAutoML(
    task = task,
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    tuning_params = {'max_tuning_time': 900},
    reader_params = {'n_jobs': N_THREADS}
)

To run autoML training use ```fit_predict``` method.

Main arguments:

- `train_data` - dataset to train.
- `roles` - column roles dict.
- `verbose` - controls the verbosity: the higher, the more messages:
        <1  : messages are not displayed;
        >=1 : the computation process for layers is displayed;
        >=2 : the information about folds processing is also displayed;
        >=3 : the hyperparameters optimization process is also displayed;
        >=4 : the training process for every algorithm is displayed;

Note: out-of-fold prediction is calculated during training and returned from the fit_predict method

In [ ]:
%%time
oof_pred = automl.fit_predict(train_df, roles = roles, verbose = 1)

[07:17:42] Start automl utilizator with listed constraints:


INFO:lightautoml.addons.utilization.utilization:Start automl utilizator with listed constraints:


[07:17:42] - time: 300.00 seconds


INFO:lightautoml.addons.utilization.utilization:- time: 300.00 seconds


[07:17:42] - CPU: 4 cores


...

[07:21:06] Model description:
Final prediction for new objects (level 0) = 
	 0.50000 * (2 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.50000 * (3 averaged models Lvl_0_Pipe_1_Mod_2_CatBoost) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 0.50000 * (2 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.50000 * (3 averaged models Lvl_0_Pipe_1_Mod_2_CatBoost) 



[07:21:06] ==================================================


INFO:lightautoml.addons.utilization.utilization:==================================================


CPU times: user 4min 57s, sys: 6.45 s, total: 5min 4s
Wall time: 3min 24s


## Prediction for test data

In [ ]:
test_pred = automl.predict(test_df)
print(f'Prediction for te_data:\n{test_pred}\nShape = {test_pred.shape}')

Prediction for te_data:
array([[0.00656395],
       [0.00816865],
       [0.00441861],
       ...,
       [0.00490397],
       [0.02964572],
       [0.01784633]], dtype=float32)
Shape = (19260, 1)


In [ ]:
print(f'OOF score: {roc_auc_score(train_df[TARGET_NAME].values, oof_pred.data[:, 0])}')

ValueError: Input contains NaN.

In [ ]:
oof_pred.data[:, 0]

array([0.44725886,        nan, 0.00581809, ..., 0.02756848,        nan,
       0.00322664], dtype=float32)

In [ ]:
np.count_nonzero(~np.isnan(oof_pred.data[:, 0]))

26964

In [ ]:
np.count_nonzero(np.isnan(oof_pred.data[:, 0])) # NaNs

17975

In [ ]:
np.count_nonzero(np.isnan(test_pred.data[:, 0])) # NaNs

0

In [ ]:
print(f'Test score: {roc_auc_score(test_df[TARGET_NAME].values, test_pred.data[:, 0])}')

Test score: 0.885788495114911


In [ ]:
test_auc = roc_auc_score(test_df[TARGET_NAME].values, test_pred.data[:, 0])
test_acc = accuracy_score(test_df[TARGET_NAME].values, test_pred.data[:, 0] > 0.5)

print("RFC test metrics: AUC={}, acc={}".format(test_auc, test_acc))

RFC test metrics: AUC=0.885788495114911, acc=0.9280373831775701


## Model analysis

In [ ]:
print(automl.create_model_str_desc())

Final prediction for new objects = 
	1.00000 * 1 averaged models with config = "conf_0_sel_type_0.yml" and different CV random_states. Their structures: 

	    Model #0.
		Final prediction for new objects (level 0) = 
			 0.11268 * (4 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
			 0.52931 * (2 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
			 0.35802 * (3 averaged models Lvl_0_Pipe_1_Mod_2_CatBoost) 






For feature importances calculation we have 2 different methods in LightAutoML:
- Fast (`fast`) - this method uses feature importances from feature selector LGBM model inside LightAutoML. It works extremely fast and almost always (almost because of situations, when feature selection is turned off or selector was removed from the final models with all GBM models). There is no need to use new labelled data.
- Accurate (`accurate`) - this method calculate *features permutation importances* for the whole LightAutoML model based on the **new labelled data**. It always works but can take a lot of time to finish (depending on the model structure, new labelled dataset size etc.).

In [ ]:
%%time

# Fast feature importances calculation
fast_fi = automl.get_feature_scores('fast')
fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (30, 10), grid = True)

AttributeError: 'NoneType' object has no attribute 'set_index'

In [ ]:
fast_fi